# Admin Startup

The purpose of this notebook is to show how to start an admin client to operate an FL experiment with a server and at least one client started.

## Prerequisites
- The [Startup notebook](Startup.ipynb) has been run successfully.
- A server and at least one client has been started.

### Check the Config Folder

The config folder should be in the `transfer/` directory:

In [1]:
import os
from IPython.display import HTML

In [2]:
workspace = 'demo_workspace/'
transfer_path = os.path.join(workspace, 'admin@nvidia.com', 'transfer')
os.listdir(transfer_path)

['spleen_example']

### Build Admin Environment and Start

In the admin folder, there is a `.whl` file which contains all the required libraries for the admin client:

In [3]:
startup_path = os.path.join(workspace, 'admin@nvidia.com', 'startup')
print("startup_path: ", startup_path)
os.listdir(startup_path)

startup_path:  demo_workspace/admin@nvidia.com/startup


['fl_admin.sh',
 'client.crt',
 'client.key',
 'readme.txt',
 'docker.sh',
 'clara_hci-4.0.0-py3-none-any.whl',
 'signature.pkl',
 'rootCA.pem']

To build the environment, you should:
- open a new terminal.
- create and then enter a new python virtual environment.
- install the `.whl` file.

To start the admin, you should:
- run `fl_admin.sh`.
- type the admin name for authentication.

In [4]:
HTML('<a href="", data-commandlinker-command="terminal:create-new"> Open a new terminal</a>')

You can accomplish these steps by running:

```
python3 -m venv venv/fl-admin
source venv/fl-admin/bin/activate
cd demo_workspace/admin@nvidia.com/startup/
pip install --upgrade pip
python3 -m pip install *.whl

bash fl_admin.sh
admin@nvidia.com
```

### Prepare for the experiment

You need to execute the following steps to prepare for the experiment:

- upload pipeline config folder
- set FL training number
- deploy the folder to client(s) and server

The commands can be:
```
upload_folder spleen_example
set_run_number 1
deploy spleen_example server
deploy spleen_example client
```

Now, let's check if the folder has been distributed into the server and all client(s):

In [5]:
run_file = 'run_1'

for name in ['server', 'org1-a', 'org1-b']:
    path = os.path.join(workspace, name, run_file)
    if os.path.exists(path):
        print("config files on {}: {}".
              format(name, os.listdir(path)))
        print(" ")

config files on server: ['mmar_server', 'mmar_org1-a', 'mmar_org1-b']
 
config files on org1-a: ['mmar_org1-a']
 
config files on org1-b: ['mmar_org1-b']
 


This example prepares two different data list files: `dataset_part1.json` and `dataset_part2.json`, and they have the same validation set and totally different training set. The default file used in `spleen_example/config_train.json` is `dataset_part1.json`. Therefore, if you want to let two clients train on different data, you can switch to use `dataset_part2.json` for `org1-b`.

[Link to org1-a config](demo_workspace/org1-a/run_1/mmar_org1-a/config/config_train.json)

[Link to org1-b config](demo_workspace/org1-b/run_1/mmar_org1-b/config/config_train.json)

### Start Training

Now, you can start training with:
```
start server
start client
```
You can check the status by running:
```
check_status server
check_status client
```

### Stop Training

You can stop training for the server and/or client(s) by running:
```
abort client
abort server
```
If you only want to stop org1-b, you can use:
```
abort client org1-b
```

### Start Training (multi-gpus)

If you would like to use multiple gpus, you can start training with n gpus (where n is the number of gpus) by running:
```
start_mgpu client n
```

The default `multi_gpu` flag is `False` in `spleen_example/config/config_train.json`, if you need to use multiple gpus, you have to change it before the `Startup` step.

### Shutdown

You can shutdown the experiment for the server and/or client(s) by running:

`shutdown client` and/or `shutdown server`

If you only want to shutdown a specific client, you can specify the client in the command like follows:
```
shutdown client fl_clients_2
```
This command will kill the client/server connection, and this command will need input of the admin name for confirmation. If you need to shutdown the server, all active clients need to be shutdown first.

After the shutdown commands, you are safe to shutdown `Startup.ipynb` to stop all containers.

### Other Commands

Please type `?` to learn more about all commands.

### Next Steps

You have now started the admin client and learnt the commands to control your FL experiment. You're now ready to create your own FL experiment!